In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from numpy import exp
from scipy import ndimage
from func import *

In [2]:
def moffat_fit(data,error,amp,x0,y0,asymmetry=False,gamma_fixed=None,alpha_fixed=None,box_size=14,MC_loops=100):
    if asymmetry:
        y, x = np.mgrid[:box_size, :box_size]
    else:
        y, x = np.mgrid[:box_size+1, :box_size+1] 
    if gamma_fixed is None and alpha_fixed is None:
        p_init = models.Moffat2D(amplitude=amp,x_0=x0,y_0=y0,gamma=4,alpha=5)
    else:
        p_init = models.Moffat2D(amp,x0,y0,gamma_fixed,alpha_fixed,fixed={'gamma':True,'alpha':True})
    f = fitting.LevMarLSQFitter()
    p = f(p_init, x, y, data)
    model = p(x,y)
    residual = data - model
    res = (residual/error)
    [amp_out,x0_out,y0_out,gamma_out,alpha_out]= p.parameters
    fwhm_out = 2*gamma_out*np.sqrt(2**(1/alpha_out)-1)
    p_parameters = np.append(p.parameters,fwhm_out)
    parameters_MC = np.zeros((len(p_parameters),MC_loops))
    for l in range(MC_loops):
        iteration_data = np.random.normal(data,error) 
        if gamma_fixed is None and alpha_fixed is None:
            p_MC_init = models.Moffat2D(amplitude=amp_out,x_0=x0_out,y_0=y0_out,gamma=gamma_out,alpha=alpha_out)
        else:
            p_MC_init = models.Moffat2D(amp_out,x0_out,y0_out,gamma_out,alpha_out,fixed={'gamma':True,'alpha':True})
        f = fitting.LevMarLSQFitter()
        p_MC = f(p_MC_init, x, y, iteration_data)
        [amp_MC,x0_MC,y0_MC,gamma_MC,alpha_MC]= p_MC.parameters
        fwhm_MC = 2*gamma_MC*np.sqrt(2**(1/alpha_MC)-1)
        p_MC_parameters = np.append(p_MC.parameters, fwhm_MC)
    parameters_MC[:,l] = p_MC_parameters    
    parameters_err = np.std(parameters_MC,1) 
    [amp_err,x0_err,y0_err,gamma_err,alpha_err,fwhm_err] = parameters_err    
    if gamma_fixed is None and alpha_fixed is None:
        (par,err) = ([amp_out,x0_out,y0_out,gamma_out,alpha_out,fwhm_out],[amp_err,x0_err,y0_err,gamma_err,alpha_err,fwhm_err])   
    else:
        (par,err) = ([amp_out,x0_out,y0_out],[amp_err,x0_err,y0_err])   
    return par,err,model,res

def maps(Hb_blr_br_data,OIII_br_data,OIII_nr_data,Hb_model,OIII_br_model,OIII_nr_model,Hb_dev,OIII_br_dev,OIII_nr_dev,obj):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(Hb_blr_br_data,name='Hb_blr_br_data'))
    hdus.append(fits.ImageHDU(OIII_br_data,name='OIII_br_data'))
    hdus.append(fits.ImageHDU(OIII_nr_data,name='OIII_nr_data'))
    hdus.append(fits.ImageHDU(Hb_model,name='Hb_blr_br_model'))
    hdus.append(fits.ImageHDU(OIII_br_model,name='OIII_br_model'))   
    hdus.append(fits.ImageHDU(OIII_nr_model,name='OIII_nr_model'))
    hdus.append(fits.ImageHDU(Hb_dev,name='Hb_blr_br_res'))
    hdus.append(fits.ImageHDU(OIII_br_dev,name='OIII_br_res'))
    hdus.append(fits.ImageHDU(OIII_nr_dev,name='OIII_nr_res'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('maps_%s.fits'%(obj),overwrite='True')
    
def moffat_table(full_data,full_error,obj):
    column_names={'amp_Hb_blr':0,'x0_Hb_Blr':1,'y0_Hb_Blr':2,'gamma':3,'alpha':4,'fwhm':5,'amp_OIII_br':6,'x0_OIII_br':7,'y0_OIII_br':8,'amp_OIII_nr':9,'x0_OIII_nr':10,'y0_OIII_nr':11}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[full_data[column_names[key]]]))
        columns.append(fits.Column(name=key+'_err',format='E',array=[full_error[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdu = fits.BinTableHDU.from_columns(coldefs)
    hdu.writeto('moffat_table_%s.fits'%(obj),overwrite=True)
    
def offset_orig(obj):
    hdu = fits.open('moffat_table_%s.fits'%(obj))
    central_tab = hdu[1].data
    central_columns = hdu[1].header
    (Hb_x,Hb_y) = (central_tab.field('x0_Hb_Blr')[0],central_tab.field('y0_Hb_Blr')[0])
    (OIII_br_x,OIII_br_y) = (central_tab.field('x0_OIII_br')[0],central_tab.field('y0_OIII_br')[0])
    (OIII_nr_x,OIII_nr_y) = (central_tab.field('x0_OIII_nr')[0],central_tab.field('y0_OIII_nr')[0])
    offset_OIII_br_pixel= [(OIII_br_x - Hb_x),(OIII_br_y - Hb_y)]
    offset_OIII_nr_pixel= [(OIII_nr_x - Hb_x),(OIII_nr_y - Hb_y)]
    return offset_OIII_br_pixel,offset_OIII_nr_pixel

In [5]:
def algorithm_script(obj,z,asymmetry):
    (Hb_blr_br_data,OIII_br_data,OIII_nr_data,amp_Hb_blr_br,amp_OIII_br,amp_OIII_nr) = loadmap('subcube_par_%s.fits'%(obj))  
    (Hb_blr_br_err,OIII_br_err,OIII_nr_err,max_err_Hb_blr_br,max_err_OIII_br,max_err_OIII_nr) = loadmap('subcube_par_err_%s.fits'%(obj))  
    
    (brightest_pixel_Hb_blr_br_x,brightest_pixel_Hb_blr_br_y,brightest_pixel_OIII_br_x,brightest_pixel_OIII_br_y,brightest_pixel_OIII_nr_x,brightest_pixel_OIII_nr_y) = brightest_pixel_flux_map(Hb_blr_br_data,OIII_br_data,OIII_nr_data) 
    
    (Hb_par,Hb_error,Hb_model,Hb_res) = moffat_fit(Hb_blr_br_data,Hb_blr_br_err,amp_Hb_blr_br,brightest_pixel_Hb_blr_br_x,brightest_pixel_Hb_blr_br_y,asymmetry,None,None,box_size=14,MC_loops=100)
    (gamma_fix,alpha_fix) = (Hb_par[3],Hb_par[4])#these two are gamma and alpha
    
    (OIII_br_par,OIII_br_error,OIII_br_model,OIII_br_res) = moffat_fit(OIII_br_data,OIII_br_err,amp_OIII_br,brightest_pixel_OIII_br_x,brightest_pixel_OIII_br_y,asymmetry,gamma_fix,alpha_fix,box_size=14,MC_loops=100)   
    (OIII_nr_par,OIII_nr_error,OIII_nr_model,OIII_nr_res) = moffat_fit(OIII_nr_data,OIII_nr_err,amp_OIII_nr,brightest_pixel_OIII_nr_x,brightest_pixel_OIII_nr_y,asymmetry,gamma_fix,alpha_fix,box_size=14,MC_loops=100)

    maps(Hb_blr_br_data,OIII_br_data,OIII_nr_data,Hb_model,OIII_br_model,OIII_nr_model,Hb_res,OIII_br_res,OIII_nr_res,obj)
    
    (full_data,full_error) = (np.append(Hb_par,[OIII_br_par,OIII_nr_par]),np.append(Hb_error,[OIII_br_error,OIII_nr_error]))
    
    moffat_table(full_data,full_error,obj)  
    
    (offset_OIII_br_pixel,offset_OIII_nr_pixel) = offset_orig(obj)
    
    #print (offset_OIII_br_pixel,offset_OIII_nr_pixel)
    
    print '%s.fits'%(obj)

In [6]:

    
z = {"HE0021-1819":0.053197,"HE0040-1105":0.041692 #,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143,"HE0253-1641":0.031588
    ,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009,"HE0433-1028":0.035550
    ,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986,"HE1029-1831":0.040261
    ,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1330-1013":0.022145,"HE1353-1917":0.035021
    ,"HE1417-0909":0.044,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

#z_remaining = {"HE2128-0221":0.05248,"HE1248-1356":0.01465}

objs = z.keys()

asymmetry = {"HE0021-1819":True,"HE0040-1105":True #,"HE0108-4743":dunno,"HE0114-0015":dunno
    ,"HE0119-0118":False,"HE0224-2834":False,"HE0227-0913":False,"HE0232-0900":False,"HE0253-1641":False
    ,"HE0345+0056":False,"HE0351+0240":False,"HE0412-0803":False,"HE0429-0247":False,"HE0433-1028":False
    ,"HE0853+0102":False,"HE0934+0119":True,"HE1011-0403":False,"HE1017-0305":False,"HE1029-1831":False
    ,"HE1107-0813":False,"HE1108-2813":True,"HE1126-0407":False,"HE1330-1013":True,"HE1353-1917":True
    ,"HE1417-0909":False,"HE2211-3903":False,"HE2222-0026":False,"HE2233+0124":False,"HE2302-0857":False}


for obj in objs:
    algorithm_script(obj,z[obj],asymmetry[obj])

HE2233+0124.fits
HE0429-0247.fits
HE0232-0900.fits
HE2222-0026.fits
HE0021-1819.fits
HE0853+0102.fits
HE1011-0403.fits
HE0040-1105.fits
HE0253-1641.fits
HE0119-0118.fits
HE0934+0119.fits
HE1017-0305.fits
HE0345+0056.fits
HE0412-0803.fits
HE0433-1028.fits
HE1330-1013.fits
HE1417-0909.fits
HE0227-0913.fits
HE1029-1831.fits
HE2302-0857.fits
HE1107-0813.fits
HE2211-3903.fits
HE1108-2813.fits
HE1126-0407.fits
HE0351+0240.fits
HE0224-2834.fits
HE1353-1917.fits


In [4]:
z = {"HE0227-0913":0.016451}
#z_remaining = {"HE2128-0221":0.05248,"HE1248-1356":0.01465}

objs = z.keys()

asymmetry = {"HE0227-0913":False}


for obj in objs:
    algorithm_script(obj,z[obj],asymmetry[obj])

([0.08071995, 0.096951485], [0.0072522163, 0.0590477])
HE0227-0913.fits


In [ ]:
offsets are in the form of (broad_OIII_offset_pixel,narrow_OIII_offset_pixel)

actual moffat offsets are ([0.08071995, 0.096951485], [0.0072522163, 0.0590477])


the centers are (x1+8,y1+8)

x1=100,y1=140  ([-0.0028338432, -0.0066108704], [-0.0011329651, -0.0062384605])
x1=58,y1=115 ([-0.0075068474, -0.01356411], [-0.00047683716, -0.007211685])
x1=156,y1=109 ([-0.006571293, -0.012761116], [-0.0009365082, -0.006652832])
x1=81,y1=130 ([0.0020051003, -0.0059518814], [-0.0010151863, -0.006995678])
x1=93,y1=120 ([-0.006237507, -0.0065956116], [-0.00051927567, -0.006494999])
x1=133,y1=152 ([-0.008470058, -0.0067601204], [0.00047683716, -0.0076389313])
x1=157,y=89 ([-0.0064082146, -0.014731884], [-0.0036568642, -0.0027785301])
x1=77,y1=74 ([-0.0028438568, -0.009002209], [-0.00052690506, -0.0057821274])
x1=113,y1=128 ([0.0017528534, -0.010829926], [-0.000644207, -0.007077694])
x1=137,y1=138 ([-0.005520344, -0.0077962875], [-0.001241684, -0.0068359375])
x1=93,y1=140 ([-0.009185314, 0.003686905], [-0.0006761551, -0.008485317])
x1=67,y1=96 ([-0.00035762787, -0.0068044662], [-0.0003027916, -0.005856514])
x1=110,y1=43 ([-0.007651329, -0.013422489], [-0.00071525574, -0.0070199966])
x1=141,y1=136 ([-0.005584717, -0.0075411797], [0.000746727, -0.0072841644])
x1=148,y1=78 ([-0.005935669, -0.008510113], [-0.0013923645, -0.0052165985])
x1=115,y1=53 ([0.0029239655, -0.016569614], [-0.0009598732, -0.0060596466])
x1=75,y1=136 ([-0.005739212, -0.006811619], [-0.0011177063, -0.006782055])
x1=107,y1=154 ([-0.0044083595, -0.011292458], [-0.0008621216, -0.006969452])
x1=41,y1=115 ([-0.0013685226, -0.0072984695], [-0.000228405, -0.0070557594])
x1=111,y1=38 ([-0.010340214, -0.0039978027], [-0.0014562607, -0.005706787])


